In [1]:
from utils import *
import os, glob
import numpy as np
import pydicom as di
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

# 3D Affine Formulae
---
\begin{align}
\mathbf{A}_{multi} = \begin{vmatrix}
F_{11}\Delta_r & F_{12}\Delta_c & \frac{T_1^1-T_1^N}{1-N} & T_1^1 \\
F_{21}\Delta_r & F_{22}\Delta_c & \frac{T_2^1-T_2^N}{1-N} & T_2^1 \\
F_{31}\Delta_r & F_{32}\Delta_c & \frac{T_3^1-T_3^N}{1-N} & T_3^1 \\
0 & 0 & 0 & 1
\end{vmatrix}
\end{align}

- $F_{ij}$ : ImageOrientationPatient 정보 
    - ($i_1$, $i_2$, $i_3$, $i_4$, $i_5$, $i_6$) = ($F_{12}$, $F_{22}$, $F_{32}$, $F_{11}$, $F_{21}$, $F_{31}$)  
    
- $\Delta_r$ : Row spacing  

- $\Delta_c$ : Col spacing  

- $T_1$ : 첫번째 DICOM 의 ImagePositionPatient 정보
    - (X, Y, Z) 형식  

- $T_N$ : 마지막 DICOM 의 ImagePositionPatient 정보
    - (X, Y, Z) 형식  
    
- $1-N$ : 1 - DICOM 갯수  



- $(X_{csv}, Y_{csv}, Z_{csv}, 1) = A_{multi} * (X,Y,Z,1)$  

- labeling 할때는 $(X,Y,Z,1) = inv(A_{multi}) * (X_{csv}, Y_{csv}, Z_{csv}, 1)$ 해줘야함.

In [4]:
to_path = './data/'

num_path = os.listdir(to_path)
num_path.remove('input')
num_path.remove('.ipynb_checkpoints')
num_path.sort()

dep=186
size= 128

data_img = None
data_lab = None

for num in num_path:
    data_path = os.listdir(to_path+num)
    
    
    for idx, data in enumerate(data_path):
        
        tmp_img = np.zeros([1, dep, size, size])
        tmp_ground = np.zeros([1, dep, size, size])
        
        if data in ['12692', '11961'] : continue
        print(data)
        img, ground, ori_coor = make_data(to_path, num, data, use_scale=False)
        diff = dep - len(img)
        
        img = resizing_data(img, size)
        
        tmp_img[0,diff:,:,:]=img
        tmp_img = (tmp_img - tmp_img.min())/(tmp_img.max() - tmp_img.min())
        
        ground = resizing_data(ground, size)
        tmp_ground[0,diff:,:,:]=ground
        
        if str(data_img)=='None':
            data_img = tmp_img
            data_lab = tmp_ground
        else:
            data_img = np.concatenate((data_img, tmp_img), axis=0)
            data_lab = np.concatenate((data_lab, tmp_ground), axis=0)

12653
12841
12575
12803
12127
11439
12459
5569
11900
12280
12422
11906
11760
7360
11534
11498
6002
6702
12259
12735


In [5]:
np.savez('./data/input/data_3d_minmax_128_186', img=data_img, lab = data_lab)

# 01
---
12575 : 171  
12653 : 170  
12692 : 156  
12803 : 160  
12841 : 156  

# 02
---
11439 : :187.dcm   (:186) 186  
11900 : 84.dcm:    (83:)  175  
11906 : :152.dcm   (:151) 151  
11961 : 25.dcm:    (24:)  167  
12127 : 25.dcm:    (24:)  156  
12280 : 25.dcm:    (24:)  139  
12422 : 25.dcm:    (24:)  156  
12459 : 25.dcm:    (24:)  163  
 5569 : 25.dcm:    (24:)  152  

# 03
---
11498 : 25.dcm:205.dcm (24:204)  180  
11534 : 89.dcm:        (88:)     175  
11760 : :185.dcm       (:184)    184  
12259 : 25.dcm:203.dcm (24:202)  178  
12735 : 25.dcm:        (24:)     154  
 6002 : 61.dcm:203.dcm (60:202)  142  
 6702 : :161.dcm       (:160)    160  
 7360 : 25.dcm:        (24:)     157  

# MAX, MIN
---

- max : 186
- min : 139
- mean : 160